# Bench

In [1]:
#!conda install sympy
#!/opt/conda/envs/cq/bin/pip install -e git+https://github.com/astanin/python-tabulate#egg=tabulate
#!/opt/conda/envs/cq/bin/pip install pint

In [2]:
import math
from fractions import Fraction

from tabulate import tabulate
#import pint
#ureg = pint.UnitRegistry()

from sympy import Rational

In [3]:
class Conf:
    FORMAT_CONTEXT = False  # Whether to apply format_context to Bench._repr_html_

def format_context(ctx):
    return Context({x: express_in(y) for (x,y) in ctx.items()})


In [4]:
class List(list):
    def __str__(self):
        return "List%s" % (list.__str__(self))

def express_in(number, tostr=True):
    _floor = math.floor(number)
    _remaining = Rational(number-_floor)
    if _floor == 0:
        _remaining = Rational(_remaining)
        if tostr:
            return '%s"' % _remaining
        else:
            return Rational(_remaining)
    if tostr:
        if _remaining:
            return '%s-%s"' % (_floor, _remaining)
        else:
            return '%s"' % (_floor)
    else:
        return (_floor, _remaining)

assert express_in(Rational("7/2"), tostr=False) == (3, Rational("1/2"))
assert express_in(0.5625, tostr=False) == Rational("9/16")


In [5]:
class Lumber2x4:
    height = 1.5 # * ureg.inches
    width = 3.5 # * ureg.inches
    
class TreatedLumber2x4:
    height = Rational("3/2") # + Rational("1/8")) # * ureg.inch  # 1 1/2
    width = (Rational("7/2") + Rational("1/8"))  # * ureg.inch  # 3 5/8
    
class TreatedLumber4x4:
    height = width = (Rational("7/2") + Rational("1/8"))  # * ureg.inch  # 3 5/8

class Context(dict):
    def set(self, key, value):
        self[key] = value
        print((key, value))
        
    def _repr_html_(self):
        return tabulate(self.items(), tablefmt='html')


"""
__divmod = divmod
def divmod(x, y):
    n, remainder = __divmod(x, y)
    if hasattr(x, 'units'):
        if not hasattr(n, 'units'):
            n = n*x.units
        if not hasattr(remainder, 'units'):
            remainder = n*x.units
    return n, remainder
""" 
    
class Bench:
    def __init__(self, length=48, width=16 + Rational("3/4"), height=20.5, short_board_length=3.5, short_board_count=4):

        ctx = self.ctx = Context()
        ctx.update(dict(length=length, width=width,
                   height=height, short_board_length=short_board_length,
                   short_board_count=short_board_count))

        boards_wide_divmod = divmod(width, TreatedLumber2x4.height)
        ctx.set('boards_wide_divmod', boards_wide_divmod)
        boards_wide = boards_wide_divmod[0] + int(bool(boards_wide_divmod[1]))
        ctx.set('boards_wide', boards_wide)
        long_board_rows = math.ceil(boards_wide / 2)
        ctx.set('long_board_rows', long_board_rows)
        short_board_rows = (boards_wide - long_board_rows)
        ctx.set('short_board_rows',  short_board_rows)
        short_boards = ((short_board_rows) * short_board_count) - 4  # leg tenons
        ctx.set('short_boards', short_boards)



        tbl = total_board_length = 0
        tbl += short_boards * short_board_length

        tbl += long_board_rows * length

        ctx.set('total_board_length', tbl)

        ctx.set('total_board_length_divmod_12', divmod(tbl, 12))

        assert 12*16 == 192
        ctx.set('total_board_length_divmod_192', divmod(tbl, 192))

        assert 12*12 == 144
        ctx.set('total_board_length_divmod_144', divmod(tbl, 144))

        legs = 4 * height
        ctx.set('leg_height', height + Rational("1/2"))
        ctx.set('legs', legs)
        ctx.set('legs_divmod_12', divmod(legs, 12))
        
        ctx.set('legs_to_shoulder', ctx['leg_height'] - (4 + Rational("1/2")))
        
        ctx.set('pieces', self.pieces())
        ctx.set('total_weight', self.calculate_weight())
        
    def pieces(self):
        return [
            [6, '2x4', 8, 17],
            [2, '4x4', 8, 38]
        ]
        # - 2x 2x4 x 16ft 
        # - 1x 2x4 x 12ft
        # - 1x 4x4 x 12

    def calculate_weight(self):
        weight = 0
        for p in self.ctx['pieces']:
            print(p)
            weight += p[0] * p[-1]
        return weight

    def __str__(self):
        return "Bench(" + str(self.ctx) + ")"
    
    __repr__ = __str__
    
    def _repr_html_(self):
        if Conf.FORMAT_CONTEXT:
            data = ((x,express_in(y) if not hasattr(y, '__iter__') else List(y)) for (x,y) in self.ctx.items())
        else:
            data = self.ctx.items()
        return tabulate(data, tablefmt='html')
    
bench = Bench()
bench

('boards_wide_divmod', (11, 1/4))
('boards_wide', 12)
('long_board_rows', 6)
('short_board_rows', 6)
('short_boards', 20)
('total_board_length', 358.000000000000)
('total_board_length_divmod_12', (29, 10.0))
('total_board_length_divmod_192', (1, 166.0))
('total_board_length_divmod_144', (2, 70.0))
('leg_height', 21.0000000000000)
('legs', 82.0)
('legs_divmod_12', (6.0, 10.0))
('legs_to_shoulder', 16.5000000000000)
('pieces', [[6, '2x4', 8, 17], [2, '4x4', 8, 38]])
[6, '2x4', 8, 17]
[2, '4x4', 8, 38]
('total_weight', 178)


length,48
width,67/4
height,20.5
short_board_length,3.5
short_board_count,4
boards_wide_divmod,"(11, 1/4)"
boards_wide,12
long_board_rows,6
short_board_rows,6
short_boards,20
total_board_length,358.000000000000


In [6]:
Conf.FORMAT_CONTEXT = True
bench

length,"48"""
width,"16-3/4"""
height,"20-1/2"""
short_board_length,"3-1/2"""
short_board_count,"4"""
boards_wide_divmod,"List[11, 1/4]"
boards_wide,"12"""
long_board_rows,"6"""
short_board_rows,"6"""
short_boards,"20"""
total_board_length,"358"""


### Steps

- [...]
- [x] Buy lumber (see: `pieces`)
  - [ ] Update lumber counts
  - [ ] Add 2x 7/16" 48" dowels = 96" (4 rails * 4 * 4" = 64")
- [x] Buy hardware
  - [x] 4x  3/8" 36" threaded rods
  - [ ] 4x rods * 4 = 16 nuts, 16 washers
    - Have 12 nuts, 12 washers
- [x] Buy tools
  - [ ] ...
  - [x] Angle grinder
  - [x] Angle grinder metal cutting disc
  - [x] Forstner bit: TODO size
  - [x] Table saw
  - [x] 4x 24"+ clamps
  - [x] Mortising router bit
  - [ ] 
- [x] Cut 4x4 legs: 4x at `height` + `height_extra` # TODO
- [x] Cut 2x4 short boards: `short_boards` x  3.5+1/8
- [x] Arrange top right-side-up
- [x] Clamp top
- [x] Let top sit clamped for awhile
- [x] Flip top upside down
- [x] Cut threaded rods to length: `bench.depth`
- [ ] Drill boards
  - [x] Make drill centering jig
  - [ ] Drill inset for threaded rods with forstner bit
  - [ ] Drill 2x inner set first, assemble, and check
  - [ ] Drill 2x outer set
- [x] Calculate side rail lengths
- [x] Calculate wide rail lengths
- [x] Cut side rails to `side_rail_length`
- [x] Cut wide rails to `wide_rail_length`
- [x] Make mortise template `TreatedLumber2x4.width`, `TreatedLumber2x4.height` centered on 4x4
- [x] Make 12x leg mortises `TreatedLumber2x4.width`, `TreatedLumber2x4.height`
  - [x] 4x mortise 1
  - [x] 4x mortise 2
  - [x] 4x mortise 3
    - [x] smaller to have a flush top shoulder
  - [x] Route each side with Mortising Bit
  - [x] Chisel square
  - [x] Chisel the rest of the way through
- [x] Cut leg tenons: `TreatedLumber2x4.width`, `TreatedLumber2x4.height`
- [x] Cut wide rail tenons such that the side rail is flush with the top of the leg shoulder
- .
- [ ] Round over edges
- [ ] Assemble base
- [ ] Assemble top
  - [ ] Glue pieces
  - [ ] Thread onto rod
    - [ ] Thread leg tenons onto rod
  - [ ] Clamp
  - [ ] Tighten bolts
- [ ] Drill holes in leg tenons for dowels
- [ ] Drive dowels


In [7]:
# calculate where to place the 
mheight = TreatedLumber2x4.width
ctx = mortise_ctx = Context(
    mortise_3_offset_top= 1,
    mortise_3_height= mheight,
    mortise_3_offset= None,
    mortise_2_height= mheight,    
    mortise_2_offset= None,    
    mortise_1_height= mheight,
    mortise_1_offset_bottom= 3.5,
)
ctx

mortise_3_offset_top,1
mortise_3_height,3.625
mortise_3_offset,
mortise_2_height,3.625
mortise_2_offset,
mortise_1_height,3.625
mortise_1_offset_bottom,3.5


In [8]:
leg_things = ctx.values()
remaining = bench.ctx['legs_to_shoulder'] - sum(x for x in ctx.values() if x is not None)
print(('remaining', remaining))
ctx['mortise_3_offset'] = ctx['mortise_2_offset'] = remaining / 2
assert sum(ctx.values()) == bench.ctx['legs_to_shoulder']
ctx['mortise_2_offset_bottom'] = sum(ctx[key] for key in ['mortise_1_offset_bottom', 'mortise_1_height', 'mortise_2_offset'])
ctx

('remaining', 1.12500000000000)


mortise_3_offset_top,1
mortise_3_height,3.625
mortise_3_offset,0.5625
mortise_2_height,3.625
mortise_2_offset,0.5625
mortise_1_height,3.625
mortise_1_offset_bottom,3.5
mortise_2_offset_bottom,7.6875


In [9]:
ctx['mortise_1_side'] = ctx['mortise_2_side'] = (TreatedLumber4x4.height - TreatedLumber2x4.height) / 2
ctx['mortise_1_side']

17/16

In [10]:
express_in(ctx['mortise_1_side'])

'1-1/16"'

In [11]:
format_context(mortise_ctx)

mortise_3_offset_top,"1"""
mortise_3_height,"3-5/8"""
mortise_3_offset,"9/16"""
mortise_2_height,"3-5/8"""
mortise_2_offset,"9/16"""
mortise_1_height,"3-5/8"""
mortise_1_offset_bottom,"3-1/2"""
mortise_2_offset_bottom,"7-11/16"""
mortise_1_side,"1-1/16"""
mortise_2_side,"1-1/16"""


In [12]:
mortise_2_from_bottom = ctx['mortise_1_offset_bottom'] + ctx['mortise_1_height'] + ctx['mortise_2_offset']
express_in(mortise_2_from_bottom)

'7-11/16"'

In [13]:
ctx['mortise_3_side'] = .75
ctx['mortise_3_width'] = TreatedLumber4x4.width - 2*ctx['mortise_3_side']
express_in(ctx['mortise_3_side']), express_in(ctx['mortise_3_width'])

('3/4"', '2-1/8"')

In [14]:
#ctx = Context()
ctx2 = Context()
ctx2['between_legs_side'] = 9 + Rational('11/16') + (2*1)
format_context(ctx2)

between_legs_side,"11-11/16"""


In [15]:
ctx = drill_jig_ctx = Context()
ctx['from_top'] = TreatedLumber4x4.width / 2
ctx['from_side_3.5_piece'] = 3.5 / 2
format_context(ctx)

from_top,"1-13/16"""
from_side_3.5_piece,"1-3/4"""


In [16]:
ctx = longboards = Context()
ctx['leg_from_side'] = 9
ctx['from_side_3.5-8_piece'] = TreatedLumber4x4.width / 2
ctx['from_side_of_leg'] = ctx['leg_from_side'] + (TreatedLumber4x4.width / 2)
format_context(ctx)

leg_from_side,"9"""
from_side_3.5-8_piece,"1-13/16"""
from_side_of_leg,"10-13/16"""


In [17]:
ctx = basectx = Context()
ctx['long_distance_btwn_outside_of_legs'] = 30
ctx['long_sticking_out'] = Rational("1/2")
ctx['long_rail_length'] = ctx['long_distance_btwn_outside_of_legs'] + (2*ctx['long_sticking_out'])

ctx['short_distance_btwn_outside_of_legs'] = 16
ctx['short_sticking_out'] = Rational("1/2")
ctx['short_rail_length'] = ctx['short_distance_btwn_outside_of_legs'] + (2*ctx['short_sticking_out'])
format_context(ctx)

long_distance_btwn_outside_of_legs,"30"""
long_sticking_out,"1/2"""
long_rail_length,"31"""
short_distance_btwn_outside_of_legs,"16"""
short_sticking_out,"1/2"""
short_rail_length,"17"""
